In [ ]:
from CraftTraining.module import CraftLightningModule
from CraftTraining.datamodule import CraftDataModule

%run CraftTraining/main_fittest.py --config CraftTraining/config/craft.yaml 

# %run CraftTraining/main.py fit --config CraftTraining/config/craft.yaml
# %run CraftTraining/main.py test --config CraftTraining/config/craft.yaml --ckpt_path best
# %run CraftTraining/main.py predict --config CraftTraining/config/craft.yaml --ckpt_path best

In [ ]:
# Visualize one batch of data and test datamodule output
"""This script visualizes one batch of images from the CraftDataModule to verify data loading and augmentation
   and prints the type and shape of the images tensor to verify correct data loading and batching."""

from CraftTraining.datamodule import CraftDataModule
from utils.craftImagevisualization import visualizeOneBatchImages

dm = CraftDataModule(
    data_dir="E:/CraftData/data",
    batch_size=3,
    num_workers=0,
    resize=512,
    gauss_cfg={"gauss_init_size": 200, "gauss_sigma": 10},
    data_cfg={
        "custom_aug": {
            "random_rotate": {"option": True, "max_angle": 45},
            "random_horizontal_flip": {"option": True}
        }
    }
)
dm.setup("fit")
batch = next(iter(dm.train_dataloader()))
images, targets, names = batch
print(type(images), images.shape)
visualizeOneBatchImages(batch)


In [ ]:
# Test CraftMetrics with synthetic data
"""this test creates synthetic ground truth and predicted polygons for multiple images,
then computes detection metrics using IoS-based matching."""

%run utils/craftmetrics.py